In [ ]:
# Block 1: Import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label, euler_number
import os
import time

In [ ]:
# Block 2: Function to process vegetation image
def process_vegetation_image(image_path, crop_size, output_size, scale_factor=0.17):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError("Image not found. Check the file path.")
    scaled_image = cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
    h, w = scaled_image.shape
    crop_w, crop_h = min(w, crop_size[0]), min(h, crop_size[1])
    start_x = (w - crop_w) // 2
    start_y = (h - crop_h) // 2
    cropped_image = scaled_image[start_y:start_y + crop_h, start_x:start_x + crop_w]
    final_image = cv2.resize(cropped_image, output_size, interpolation=cv2.INTER_LINEAR)
    return final_image

In [ ]:
# Block 3: Function to compute Euler characteristic
def compute_euler_characteristic(binary_image):
    labeled_img = label(binary_image, connectivity=2)
    return euler_number(labeled_img)

In [ ]:
# Block 4: Function to compute Euler curve for an image
def compute_euler_curve(processed_image):
    euler_values = []
    for threshold in range(256):
        _, binary = cv2.threshold(processed_image, threshold, 255, cv2.THRESH_BINARY)
        euler_values.append(compute_euler_characteristic(binary))
    return euler_values

In [ ]:
# Block 5: Function to plot and save the Euler characteristic curve
def save_euler_plot(euler_values, image_name, output_folder):
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    os.makedirs(output_folder, exist_ok=True)
    graph_path = os.path.join(output_folder, f"euler_curve_{image_name}_{timestamp}.png")
    plt.figure(figsize=(8, 5))
    plt.plot(range(256), euler_values, marker='o', linestyle='-', color='b')
    plt.title(f"Euler Characteristic Curve: {image_name}")
    plt.xlabel("Threshold")
    plt.ylabel("Euler Characteristic")
    plt.grid(True)
    plt.savefig(graph_path, dpi=300)
    plt.show()
    print(f"✅ Saved graph to: {graph_path}")

In [ ]:
# Block 6: Process all images in the 'images' folder
def analyze_all_images(input_folder, crop_size, output_size, output_folder):
    for file in os.listdir(input_folder):
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif')):
            image_path = os.path.join(input_folder, file)
            print(f"Processing {image_path}...")
            try:
                processed = process_vegetation_image(image_path, crop_size, output_size)
                euler_values = compute_euler_curve(processed)
                save_euler_plot(euler_values, os.path.splitext(file)[0], output_folder)
            except Exception as e:
                print(f"❌ Failed to process {file}: {e}")

In [ ]:
# Block 7: Run the analysis
input_folder = "./images"
output_folder = "./output"
analyze_all_images(input_folder, crop_size=(50, 50), output_size=(50, 50), output_folder=output_folder)